**l'entrainement aprés normalization de Zak Dataset**

In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

Chargement du Dataset

In [5]:
import pandas as pd
df=pd.read_csv("/kaggle/input/new-dataset/MyDataset.csv")

In [6]:
print(df['Label'].value_counts())

Label
hate      9509
normal    8698
Name: count, dtype: int64


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split


# Convertir les labels en numérique
label_mapping = {"hate": 1, "normal": 0}
df["Label"] = df["Label"].map(label_mapping)


train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["Text"], df["Label"], test_size=0.2, random_state=42
)

tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")

def tokenize_data(texts):
    return tokenizer(list(texts), padding=True, truncation=True, max_length=512)

train_encodings = tokenize_data(train_texts)
test_encodings = tokenize_data(test_texts)

tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

# Convertir en dataset PyTorch
train_dataset = HateSpeechDataset(train_encodings, train_labels.tolist())
test_dataset = HateSpeechDataset(test_encodings, test_labels.tolist())

In [9]:
from sklearn.metrics import accuracy_score
from transformers import EarlyStoppingCallback

model = AutoModelForSequenceClassification.from_pretrained("aubmindlab/bert-base-arabertv02", num_labels=2)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none"
)

early_stopping = EarlyStoppingCallback(early_stopping_patience=2)  

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.288400,0.236422,0.924492
2,0.175100,0.174897,0.939319
3,0.112800,0.239746,0.944811
4,0.072900,0.307726,0.943712


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=3644, training_loss=0.15503026206459355, metrics={'train_runtime': 1146.107, 'train_samples_per_second': 63.541, 'train_steps_per_second': 3.974, 'total_flos': 3203490154539600.0, 'train_loss': 0.15503026206459355, 'epoch': 4.0})

In [11]:
#  Évaluer le modèle sur les données de test
results = trainer.evaluate()
print("✅ Résultats de l'évaluation :", results)

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


✅ Résultats de l'évaluation : {'eval_loss': 0.1748967319726944, 'eval_accuracy': 0.9393190554640307, 'eval_runtime': 17.4859, 'eval_samples_per_second': 208.282, 'eval_steps_per_second': 13.039, 'epoch': 4.0}


In [13]:
from sklearn.metrics import classification_report
predictions = trainer.predict(test_dataset)
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(axis=-1)
                                                            
print(classification_report(y_true, y_pred))

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.94      0.93      0.93      1685
           1       0.94      0.95      0.94      1957

    accuracy                           0.94      3642
   macro avg       0.94      0.94      0.94      3642
weighted avg       0.94      0.94      0.94      3642



Testing

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

def predict(text):
    model.eval()
    
    
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()

   
    label_mapping = {0: "normal", 1: "hate"} 
    return label_mapping.get(prediction, "inconnu")


In [23]:
text_test= "الأفارقة مكانهم في الغابة، مش في شوارعنا"
print("Prédiction :", predict(text_test))

Prédiction : hate


Enregistrement du modèle

In [18]:
torch.save(model, 'HateSpeechModel1.pth') 


In [24]:

model.save_pretrained('HateSpeechModel2')
